In [2]:
!pip install gdown

import gdown

# Define the URL of the file to download
url = 'https://drive.google.com/uc?id=1F_FNJKOE1oOZPqdLIW0Wt7LoC2InDnjV'

# Define the output file path
output = '/content/data.csv'

# Download the file using gdown
gdown.download(url, output, quiet=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1F_FNJKOE1oOZPqdLIW0Wt7LoC2InDnjV
To: /content/data.csv
100%|██████████| 253M/253M [00:03<00:00, 64.4MB/s]


'/content/data.csv'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
train_data = pd.read_csv('/content/data.csv')

In [5]:
train_data.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,kfold,text
0,1505480,"Select settings [100 COUNT] Microwaveable, com...","[INCLUDES: 100 light brown compostable bowls.,...",Your dinnerware matters. It helps to define an...,474,2108,0,select setting 100 count microwaveable compost...
1,2447219,The Customer Service In Africa: How To Improve...,,,6142,12,0,customer service africa improve customer exper...
2,369455,Brentwood (Images of America),,,223,229,0,brentwood image america
3,404896,Claire's Sketch Pad: 50 Blank Pages to draw an...,,,10,81,0,claires sketch pad 50 blank page draw anything...
4,463483,2011/12 Boston Restaurants (Zagat Survey Bosto...,,,12316,68,0,201112 boston restaurant zagat survey boston r...


In [6]:
train_data.shape

(200000, 8)

**Outlier Removal**

In [7]:
from sklearn.impute import SimpleImputer
numeric_imputer = SimpleImputer(strategy='median')
train_data['PRODUCT_LENGTH'] = numeric_imputer.fit_transform(train_data['PRODUCT_LENGTH'].values.reshape(-1, 1))

**Embedding**

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from sklearn.metrics import classification_report

In [ ]:
text_data = train_data['text'].astype(str).str.lower().str.split()  # Convert text to lowercase and tokenize

embedding_size = 100 
window_size = 5 
model = Word2Vec(text_data, vector_size=embedding_size, window=window_size, min_count=1)

def get_word_embeddings(word):
    if word in model.wv:
        return model.wv[word]
    else:
        return np.zeros(embedding_size) 
text_features = text_data.apply(lambda x: [get_word_embeddings(word) for word in x])

In [ ]:
text_features_avg = text_features.apply(lambda x: np.mean(x, axis=0))
text_features_avg = np.stack(text_features_avg)

In [ ]:
print(type(text_features_avg))

<class 'numpy.ndarray'>


In [ ]:
column_names = ['PRODUCT_ID', 'PRODUCT_TYPE_ID']  
X = train_data[column_names]
Y = train_data['PRODUCT_LENGTH']

In [ ]:
X_ = X.values
print(X_.shape)
X_combined = np.hstack(( X_ ,text_features_avg))
print(X_combined.shape)
Y_ = Y.values

(200000, 2)
(200000, 102)


**Ensembling 4 knn model using Bagging Classifier**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(text_features_avg,Y_, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=25) 
bagging = BaggingClassifier(base_estimator=knn, n_estimators=2, random_state=42)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Ensemble Accuracy: 0.183875
